In [ ]:
# 1. Setup and Data Verification
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Define paths
TRAIN_DIR = 'FER-2013/train'
TEST_DIR = 'FER-2013/test'

# Define emotion classes
EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

# Verify directory structure and count images
def verify_dataset():
    """Verify the dataset structure and count images per class"""
    print("=" * 60)
    print("DATASET VERIFICATION")
    print("=" * 60)
    
    train_counts = {}
    test_counts = {}
    
    # Check train directory
    print("\n📁 TRAINING DATA:")
    print("-" * 60)
    for emotion in EMOTIONS:
        train_path = os.path.join(TRAIN_DIR, emotion)
        if os.path.exists(train_path):
            count = len([f for f in os.listdir(train_path) if f.endswith(('.jpg', '.png', '.jpeg'))])
            train_counts[emotion] = count
            print(f"  {emotion:10s}: {count:5d} images")
        else:
            print(f"  {emotion:10s}: MISSING FOLDER ⚠️")
            train_counts[emotion] = 0
    
    print(f"\n  {'TOTAL':10s}: {sum(train_counts.values()):5d} images")
    
    # Check test directory
    print("\n📁 TEST DATA:")
    print("-" * 60)
    for emotion in EMOTIONS:
        test_path = os.path.join(TEST_DIR, emotion)
        if os.path.exists(test_path):
            count = len([f for f in os.listdir(test_path) if f.endswith(('.jpg', '.png', '.jpeg'))])
            test_counts[emotion] = count
            print(f"  {emotion:10s}: {count:5d} images")
        else:
            print(f"  {emotion:10s}: MISSING FOLDER ⚠️")
            test_counts[emotion] = 0
    
    print(f"\n  {'TOTAL':10s}: {sum(test_counts.values()):5d} images")
    print("=" * 60)
    
    return train_counts, test_counts

# Display sample images from each class
def display_samples(data_dir, emotions, samples_per_class=3):
    """Display sample images from each emotion class"""
    fig, axes = plt.subplots(len(emotions), samples_per_class, 
                            figsize=(12, 2*len(emotions)))
    fig.suptitle(f'Sample Images from {data_dir}', fontsize=16, fontweight='bold')
    
    for i, emotion in enumerate(emotions):
        emotion_path = os.path.join(data_dir, emotion)
        if not os.path.exists(emotion_path):
            continue
            
        # Get list of images
        images = [f for f in os.listdir(emotion_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
        
        # Sample random images
        sample_images = np.random.choice(images, min(samples_per_class, len(images)), replace=False)
        
        for j, img_name in enumerate(sample_images):
            img_path = os.path.join(emotion_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            axes[i, j].imshow(img, cmap='gray')
            axes[i, j].axis('off')
            
            if j == 0:
                axes[i, j].set_title(f'{emotion}\n{img.shape}', fontweight='bold')
            else:
                axes[i, j].set_title(f'{img.shape}')
    
    plt.tight_layout()
    plt.show()

# Run verification
train_counts, test_counts = verify_dataset()

# Display samples
print("\nDisplaying sample images from training set...")
display_samples(TRAIN_DIR, EMOTIONS, samples_per_class=3)

# Create summary dataframe
summary_df = pd.DataFrame({
    'Emotion': EMOTIONS,
    'Train Count': [train_counts.get(e, 0) for e in EMOTIONS],
    'Test Count': [test_counts.get(e, 0) for e in EMOTIONS]
})
summary_df['Total'] = summary_df['Train Count'] + summary_df['Test Count']
summary_df['Train %'] = (summary_df['Train Count'] / summary_df['Train Count'].sum() * 100).round(2)

print("\n" + "=" * 60)
print("DATASET SUMMARY")
print("=" * 60)
print(summary_df.to_string(index=False))
print("=" * 60)

In [ ]:
# Milestone 1: Face Detection & Visual Check
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Option 1: Haar Cascade Face Detector
class HaarFaceDetector:
    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
    
    def detect(self, image):
        """Detect faces in grayscale image"""
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        
        faces = self.face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )
        
        return faces

# Option 2: MediaPipe Face Detection
try:
    import mediapipe as mp
    
    class MediaPipeFaceDetector:
        def __init__(self):
            self.mp_face_detection = mp.solutions.face_detection
            self.face_detection = self.mp_face_detection.FaceDetection(
                model_selection=0,  # 0 for short-range detection
                min_detection_confidence=0.5
            )
        
        def detect(self, image):
            """Detect faces using MediaPipe"""
            if len(image.shape) == 2:
                # Convert grayscale to RGB
                rgb_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            else:
                rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            results = self.face_detection.process(rgb_image)
            
            if not results.detections:
                return np.array([])
            
            # Convert MediaPipe detections to bounding boxes
            h, w = image.shape[:2]
            faces = []
            
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                x = int(bbox.xmin * w)
                y = int(bbox.ymin * h)
                width = int(bbox.width * w)
                height = int(bbox.height * h)
                faces.append([x, y, width, height])
            
            return np.array(faces)
    
    MEDIAPIPE_AVAILABLE = True
except ImportError:
    MEDIAPIPE_AVAILABLE = False
    print("MediaPipe not available, using Haar Cascade only")

# Initialize detector (prefer Haar for FER2013 as faces are already cropped)
detector = HaarFaceDetector()
print("✓ Face detector initialized (Haar Cascade)")

def draw_faces(image, faces, color=(0, 255, 0), thickness=2):
    """Draw bounding boxes on detected faces"""
    result = image.copy()
    if len(result.shape) == 2:
        result = cv2.cvtColor(result, cv2.COLOR_GRAY2BGR)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(result, (x, y), (x+w, y+h), color, thickness)
        cv2.putText(result, f'{w}x{h}', (x, y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    
    return result

def test_face_detection(data_dir, emotions, samples_per_class=4):
    """Test face detection on samples from each emotion class"""
    fig, axes = plt.subplots(len(emotions), samples_per_class, 
                            figsize=(15, 2.5*len(emotions)))
    fig.suptitle('Face Detection Results (Green boxes = detected faces)', 
                fontsize=16, fontweight='bold')
    
    detection_stats = {}
    
    for i, emotion in enumerate(emotions):
        emotion_path = os.path.join(data_dir, emotion)
        if not os.path.exists(emotion_path):
            continue
        
        images = [f for f in os.listdir(emotion_path) 
                 if f.endswith(('.jpg', '.png', '.jpeg'))]
        sample_images = np.random.choice(images, 
                                        min(samples_per_class, len(images)), 
                                        replace=False)
        
        detected = 0
        for j, img_name in enumerate(sample_images):
            img_path = os.path.join(emotion_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            # Detect faces
            faces = detector.detect(img)
            
            # Draw boxes
            result = draw_faces(img, faces)
            
            # Display
            axes[i, j].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
            axes[i, j].axis('off')
            
            # Title with detection info
            if len(faces) > 0:
                detected += 1
                title = f'✓ {len(faces)} face(s)'
            else:
                title = '✗ No face'
            
            if j == 0:
                axes[i, j].set_title(f'{emotion}\n{title}', fontweight='bold')
            else:
                axes[i, j].set_title(title)
        
        detection_stats[emotion] = f"{detected}/{samples_per_class}"
    
    plt.tight_layout()
    plt.show()
    
    # Print detection statistics
    print("\n" + "=" * 60)
    print("FACE DETECTION STATISTICS (Sample)")
    print("=" * 60)
    for emotion, stat in detection_stats.items():
        print(f"  {emotion:10s}: {stat} detected")
    print("=" * 60)
    
    return detection_stats

# Run face detection test
print("\nTesting face detection on training samples...")
detection_stats = test_face_detection(TRAIN_DIR, EMOTIONS, samples_per_class=4)

# Analyze detection rate across entire dataset
def analyze_detection_rate(data_dir, emotions, max_samples=200):
    """Analyze face detection rate across the dataset"""
    print("\n" + "=" * 60)
    print("DETECTION RATE ANALYSIS")
    print("=" * 60)
    
    results = {}
    
    for emotion in emotions:
        emotion_path = os.path.join(data_dir, emotion)
        if not os.path.exists(emotion_path):
            continue
        
        images = [f for f in os.listdir(emotion_path) 
                 if f.endswith(('.jpg', '.png', '.jpeg'))]
        
        # Sample subset if too many images
        if len(images) > max_samples:
            images = np.random.choice(images, max_samples, replace=False)
        
        detected_count = 0
        total_faces = 0
        
        for img_name in images:
            img_path = os.path.join(emotion_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            faces = detector.detect(img)
            
            if len(faces) > 0:
                detected_count += 1
                total_faces += len(faces)
        
        detection_rate = (detected_count / len(images)) * 100
        avg_faces = total_faces / detected_count if detected_count > 0 else 0
        
        results[emotion] = {
            'total': len(images),
            'detected': detected_count,
            'rate': detection_rate,
            'avg_faces': avg_faces
        }
        
        print(f"  {emotion:10s}: {detected_count:3d}/{len(images):3d} "
              f"({detection_rate:.1f}%) | Avg faces/img: {avg_faces:.2f}")
    
    print("=" * 60)
    
    return results

# Analyze detection rate
detection_analysis = analyze_detection_rate(TRAIN_DIR, EMOTIONS, max_samples=200)

In [ ]:
# Milestone 2 - Track A: Landmark Feature Extraction
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm
import pickle

class LandmarkFeatureExtractor:
    def __init__(self):
        """Initialize MediaPipe FaceMesh for landmark detection"""
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh = self.mp_face_mesh.FaceMesh(
            static_image_mode=True,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5
        )
        
    def crop_largest_face(self, image):
        """Crop the largest detected face from the image"""
        faces = detector.detect(image)
        
        if len(faces) == 0:
            # If no face detected, return the center crop
            h, w = image.shape[:2]
            size = min(h, w)
            y = (h - size) // 2
            x = (w - size) // 2
            return image[y:y+size, x:x+size]
        
        # Get largest face
        largest_face = faces[np.argmax([w*h for (x, y, w, h) in faces])]
        x, y, w, h = largest_face
        
        # Add some padding
        padding = int(0.1 * min(w, h))
        x = max(0, x - padding)
        y = max(0, y - padding)
        w = min(image.shape[1] - x, w + 2*padding)
        h = min(image.shape[0] - y, h + 2*padding)
        
        return image[y:y+h, x:x+w]
    
    def extract_landmarks(self, image):
        """Extract facial landmarks from image"""
        # Crop face
        face_crop = self.crop_largest_face(image)
        
        # Convert to RGB
        if len(face_crop.shape) == 2:
            rgb_image = cv2.cvtColor(face_crop, cv2.COLOR_GRAY2RGB)
        else:
            rgb_image = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
        
        # Process with FaceMesh
        results = self.face_mesh.process(rgb_image)
        
        if not results.multi_face_landmarks:
            return None
        
        # Get landmarks
        landmarks = results.multi_face_landmarks[0]
        
        # Extract coordinates
        h, w = rgb_image.shape[:2]
        coords = []
        for landmark in landmarks.landmark:
            coords.append([landmark.x * w, landmark.y * h])
        
        return np.array(coords)
    
    def normalize_landmarks(self, landmarks):
        """Normalize landmarks by centering and scaling"""
        if landmarks is None:
            return None
        
        # Center landmarks
        centroid = np.mean(landmarks, axis=0)
        centered = landmarks - centroid
        
        # Calculate inter-pupillary distance for scaling
        # Left eye: landmark 33, Right eye: landmark 263
        left_eye = landmarks[33]
        right_eye = landmarks[263]
        ipd = np.linalg.norm(left_eye - right_eye)
        
        if ipd < 1e-6:  # Avoid division by zero
            ipd = 1.0
        
        # Scale by IPD
        normalized = centered / ipd
        
        return normalized
    
    def compute_geometric_features(self, landmarks):
        """Compute geometric features from landmarks"""
        if landmarks is None:
            return None
        
        # Eye Aspect Ratio (EAR) - average for both eyes
        def eye_aspect_ratio(eye_points):
            # Vertical distances
            v1 = np.linalg.norm(eye_points[1] - eye_points[5])
            v2 = np.linalg.norm(eye_points[2] - eye_points[4])
            # Horizontal distance
            h = np.linalg.norm(eye_points[0] - eye_points[3])
            ear = (v1 + v2) / (2.0 * h)
            return ear
        
        # Left eye landmarks (simplified)
        left_eye = landmarks[[33, 160, 158, 133, 153, 144]]
        # Right eye landmarks
        right_eye = landmarks[[362, 385, 387, 263, 373, 380]]
        
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        avg_ear = (left_ear + right_ear) / 2.0
        
        # Mouth Aspect Ratio (MAR)
        # Upper lip: 13, Lower lip: 14, Left corner: 78, Right corner: 308
        mouth_top = landmarks[13]
        mouth_bottom = landmarks[14]
        mouth_left = landmarks[78]
        mouth_right = landmarks[308]
        
        mouth_height = np.linalg.norm(mouth_top - mouth_bottom)
        mouth_width = np.linalg.norm(mouth_left - mouth_right)
        mar = mouth_height / (mouth_width + 1e-6)
        
        return np.array([avg_ear, left_ear, right_ear, mar])
    
    def extract_features(self, image):
        """Extract complete feature vector from image"""
        # Get landmarks
        landmarks = self.extract_landmarks(image)
        
        if landmarks is None:
            return None
        
        # Normalize landmarks
        normalized = self.normalize_landmarks(landmarks)
        
        # Flatten normalized landmarks
        flattened_landmarks = normalized.flatten()
        
        # Compute geometric features
        geometric_features = self.compute_geometric_features(landmarks)
        
        # Concatenate all features
        features = np.concatenate([flattened_landmarks, geometric_features])
        
        return features

# Initialize feature extractor
print("Initializing Landmark Feature Extractor...")
landmark_extractor = LandmarkFeatureExtractor()
print("✓ Landmark Feature Extractor initialized")

# Test on a few samples
def test_landmark_extraction(data_dir, emotions, samples=2):
    """Test landmark extraction on sample images"""
    fig, axes = plt.subplots(len(emotions), samples*2, 
                            figsize=(12, 2*len(emotions)))
    fig.suptitle('Landmark Extraction Test', fontsize=16, fontweight='bold')
    
    for i, emotion in enumerate(emotions):
        emotion_path = os.path.join(data_dir, emotion)
        if not os.path.exists(emotion_path):
            continue
        
        images = [f for f in os.listdir(emotion_path) 
                 if f.endswith(('.jpg', '.png', '.jpeg'))]
        sample_images = np.random.choice(images, min(samples, len(images)), 
                                        replace=False)
        
        for j, img_name in enumerate(sample_images):
            img_path = os.path.join(emotion_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            # Extract landmarks
            landmarks = landmark_extractor.extract_landmarks(img)
            
            # Original image
            axes[i, j*2].imshow(img, cmap='gray')
            axes[i, j*2].axis('off')
            if j == 0:
                axes[i, j*2].set_title(f'{emotion}\nOriginal', fontweight='bold')
            else:
                axes[i, j*2].set_title('Original')
            
            # Image with landmarks
            face_crop = landmark_extractor.crop_largest_face(img)
            if len(face_crop.shape) == 2:
                vis_img = cv2.cvtColor(face_crop, cv2.COLOR_GRAY2BGR)
            else:
                vis_img = face_crop.copy()
            
            if landmarks is not None:
                for (x, y) in landmarks:
                    cv2.circle(vis_img, (int(x), int(y)), 1, (0, 255, 0), -1)
                title = f'✓ {len(landmarks)} landmarks'
            else:
                title = '✗ No landmarks'
            
            axes[i, j*2+1].imshow(cv2.cvtColor(vis_img, cv2.COLOR_BGR2RGB))
            axes[i, j*2+1].axis('off')
            axes[i, j*2+1].set_title(title)
    
    plt.tight_layout()
    plt.show()

print("\nTesting landmark extraction...")
test_landmark_extraction(TRAIN_DIR, EMOTIONS, samples=2)

# Extract features for entire dataset
def extract_dataset_features(data_dir, emotions, extractor, save_path=None):
    """Extract features from entire dataset"""
    X = []
    y = []
    failed_count = 0
    
    print(f"\nExtracting features from {data_dir}...")
    
    for emotion_idx, emotion in enumerate(emotions):
        emotion_path = os.path.join(data_dir, emotion)
        if not os.path.exists(emotion_path):
            continue
        
        images = [f for f in os.listdir(emotion_path) 
                 if f.endswith(('.jpg', '.png', '.jpeg'))]
        
        print(f"  Processing {emotion}... ({len(images)} images)")
        
        for img_name in tqdm(images, desc=f"  {emotion}", leave=False):
            img_path = os.path.join(emotion_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            features = extractor.extract_features(img)
            
            if features is not None:
                X.append(features)
                y.append(emotion_idx)
            else:
                failed_count += 1
    
    X = np.array(X)
    y = np.array(y)
    
    print(f"\n✓ Extracted {len(X)} feature vectors")
    print(f"  Feature dimension: {X.shape[1]}")
    print(f"  Failed extractions: {failed_count}")
    
    if save_path:
        np.savez(save_path, X=X, y=y, emotions=emotions)
        print(f"  Saved to: {save_path}")
    
    return X, y

# Extract features from train and test sets
print("\n" + "="*60)
print("EXTRACTING LANDMARK FEATURES")
print("="*60)

X_train_landmarks, y_train = extract_dataset_features(
    TRAIN_DIR, EMOTIONS, landmark_extractor, 
    save_path='features_train_landmarks.npz'
)

X_test_landmarks, y_test = extract_dataset_features(
    TEST_DIR, EMOTIONS, landmark_extractor,
    save_path='features_test_landmarks.npz'
)

print("\n" + "="*60)
print("LANDMARK FEATURE EXTRACTION COMPLETE")
print("="*60)
print(f"Train set: {X_train_landmarks.shape}")
print(f"Test set:  {X_test_landmarks.shape}")
print("="*60)

In [ ]:
# Milestone 2 - Track B: CNN Deep Feature Extraction
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model
import cv2
import numpy as np
from tqdm import tqdm

class CNNFeatureExtractor:
    def __init__(self, target_size=(224, 224)):
        """Initialize pretrained CNN for feature extraction"""
        self.target_size = target_size
        
        # Load pretrained MobileNetV2 without top layer
        print("Loading pretrained MobileNetV2...")
        base_model = MobileNetV2(
            weights='imagenet',
            include_top=False,
            input_shape=(*target_size, 3),
            pooling='avg'  # Global average pooling
        )
        
        # Use the model as feature extractor
        self.model = Model(inputs=base_model.input, outputs=base_model.output)
        
        # Freeze the model (no training)
        self.model.trainable = False
        
        print(f"✓ CNN Feature Extractor initialized")
        print(f"  Output feature dimension: {self.model.output_shape[1]}")
    
    def crop_largest_face(self, image):
        """Crop the largest detected face from the image"""
        faces = detector.detect(image)
        
        if len(faces) == 0:
            # If no face detected, return the center crop
            h, w = image.shape[:2]
            size = min(h, w)
            y = (h - size) // 2
            x = (w - size) // 2
            return image[y:y+size, x:x+size]
        
        # Get largest face
        largest_face = faces[np.argmax([w*h for (x, y, w, h) in faces])]
        x, y, w, h = largest_face
        
        # Add some padding
        padding = int(0.1 * min(w, h))
        x = max(0, x - padding)
        y = max(0, y - padding)
        w = min(image.shape[1] - x, w + 2*padding)
        h = min(image.shape[0] - y, h + 2*padding)
        
        return image[y:y+h, x:x+w]
    
    def preprocess_image(self, image):
        """Preprocess image for CNN input"""
        # Crop face
        face_crop = self.crop_largest_face(image)
        
        # Resize to target size
        resized = cv2.resize(face_crop, self.target_size)
        
        # Convert grayscale to 3 channels
        if len(resized.shape) == 2:
            rgb_image = cv2.cvtColor(resized, cv2.COLOR_GRAY2RGB)
        else:
            rgb_image = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
        
        return rgb_image
    
    def extract_features(self, image):
        """Extract deep features from a single image"""
        # Preprocess
        processed = self.preprocess_image(image)
        
        # Add batch dimension and preprocess for MobileNetV2
        batch = np.expand_dims(processed, axis=0)
        batch = preprocess_input(batch)
        
        # Extract features
        features = self.model.predict(batch, verbose=0)
        
        return features[0]  # Remove batch dimension
    
    def extract_features_batch(self, images, batch_size=32):
        """Extract features from multiple images efficiently"""
        # Preprocess all images
        processed_images = []
        for img in images:
            processed = self.preprocess_image(img)
            processed_images.append(processed)
        
        # Convert to array and preprocess
        batch = np.array(processed_images)
        batch = preprocess_input(batch)
        
        # Extract features in batches
        features = self.model.predict(batch, batch_size=batch_size, verbose=0)
        
        return features

# Initialize CNN feature extractor
print("\n" + "="*60)
print("INITIALIZING CNN FEATURE EXTRACTOR")
print("="*60)
cnn_extractor = CNNFeatureExtractor(target_size=(224, 224))
print("="*60)

# Test on a few samples
def test_cnn_extraction(data_dir, emotions, samples=2):
    """Test CNN feature extraction on sample images"""
    fig, axes = plt.subplots(len(emotions), samples*2, 
                            figsize=(12, 2*len(emotions)))
    fig.suptitle('CNN Feature Extraction Test', fontsize=16, fontweight='bold')
    
    for i, emotion in enumerate(emotions):
        emotion_path = os.path.join(data_dir, emotion)
        if not os.path.exists(emotion_path):
            continue
        
        images = [f for f in os.listdir(emotion_path) 
                 if f.endswith(('.jpg', '.png', '.jpeg'))]
        sample_images = np.random.choice(images, min(samples, len(images)), 
                                        replace=False)
        
        for j, img_name in enumerate(sample_images):
            img_path = os.path.join(emotion_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            # Extract features
            features = cnn_extractor.extract_features(img)
            
            # Original image
            axes[i, j*2].imshow(img, cmap='gray')
            axes[i, j*2].axis('off')
            if j == 0:
                axes[i, j*2].set_title(f'{emotion}\nOriginal (48x48)', 
                                      fontweight='bold')
            else:
                axes[i, j*2].set_title('Original (48x48)')
            
            # Preprocessed image
            preprocessed = cnn_extractor.preprocess_image(img)
            axes[i, j*2+1].imshow(preprocessed)
            axes[i, j*2+1].axis('off')
            axes[i, j*2+1].set_title(f'CNN Input (224x224)\n✓ {len(features)} features')
    
    plt.tight_layout()
    plt.show()

print("\nTesting CNN feature extraction...")
test_cnn_extraction(TRAIN_DIR, EMOTIONS, samples=2)

# Extract features for entire dataset
def extract_dataset_cnn_features(data_dir, emotions, extractor, 
                                 batch_size=32, save_path=None):
    """Extract CNN features from entire dataset"""
    X = []
    y = []
    
    print(f"\nExtracting CNN features from {data_dir}...")
    
    for emotion_idx, emotion in enumerate(emotions):
        emotion_path = os.path.join(data_dir, emotion)
        if not os.path.exists(emotion_path):
            continue
        
        image_files = [f for f in os.listdir(emotion_path) 
                      if f.endswith(('.jpg', '.png', '.jpeg'))]
        
        print(f"  Processing {emotion}... ({len(image_files)} images)")
        
        # Process in batches for efficiency
        for i in tqdm(range(0, len(image_files), batch_size), 
                     desc=f"  {emotion}", leave=False):
            batch_files = image_files[i:i+batch_size]
            batch_images = []
            
            for img_name in batch_files:
                img_path = os.path.join(emotion_path, img_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                batch_images.append(img)
            
            # Extract features for batch
            batch_features = extractor.extract_features_batch(batch_images, 
                                                             batch_size=len(batch_images))
            
            X.extend(batch_features)
            y.extend([emotion_idx] * len(batch_features))
    
    X = np.array(X)
    y = np.array(y)
    
    print(f"\n✓ Extracted {len(X)} feature vectors")
    print(f"  Feature dimension: {X.shape[1]}")
    
    if save_path:
        np.savez(save_path, X=X, y=y, emotions=emotions)
        print(f"  Saved to: {save_path}")
    
    return X, y

# Extract features from train and test sets
print("\n" + "="*60)
print("EXTRACTING CNN FEATURES")
print("="*60)

X_train_cnn, y_train_cnn = extract_dataset_cnn_features(
    TRAIN_DIR, EMOTIONS, cnn_extractor, 
    batch_size=32,
    save_path='features_train_cnn.npz'
)

X_test_cnn, y_test_cnn = extract_dataset_cnn_features(
    TEST_DIR, EMOTIONS, cnn_extractor,
    batch_size=32,
    save_path='features_test_cnn.npz'
)

print("\n" + "="*60)
print("CNN FEATURE EXTRACTION COMPLETE")
print("="*60)
print(f"Train set: {X_train_cnn.shape}")
print(f"Test set:  {X_test_cnn.shape}")
print("="*60)

# Verify labels match
assert np.array_equal(y_train, y_train_cnn), "Train labels mismatch!"
assert np.array_equal(y_test, y_test_cnn), "Test labels mismatch!"
print("✓ Labels verified - all sets aligned")

In [ ]:
# Model Training - Both Tracks
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
import time
import pickle

# Set random seed for reproducibility
RANDOM_SEED = 42

print("\n" + "="*60)
print("MODEL TRAINING")
print("="*60)

# Dictionary to store all results
results = {}

# ============================================================================
# TRACK A: LANDMARK FEATURES
# ============================================================================

print("\n" + "-"*60)
print("TRACK A: LANDMARK FEATURES")
print("-"*60)

# Standardize features for landmark-based models
print("\nStandardizing landmark features...")
scaler_landmarks = StandardScaler()
X_train_landmarks_scaled = scaler_landmarks.fit_transform(X_train_landmarks)
X_test_landmarks_scaled = scaler_landmarks.transform(X_test_landmarks)
print("✓ Features standardized")

# Save scaler
with open('scaler_landmarks.pkl', 'wb') as f:
    pickle.dump(scaler_landmarks, f)

# Model 1: SVM with RBF kernel (Landmark features)
print("\n[1/4] Training SVM on Landmark features...")
print("  Hyperparameters: C=1.0, kernel='rbf', gamma='scale'")
start_time = time.time()

svm_landmarks = SVC(
    C=1.0,
    kernel='rbf',
    gamma='scale',
    random_state=RANDOM_SEED,
    verbose=False
)
svm_landmarks.fit(X_train_landmarks_scaled, y_train)

train_time = time.time() - start_time
y_pred_train = svm_landmarks.predict(X_train_landmarks_scaled)
y_pred_test = svm_landmarks.predict(X_test_landmarks_scaled)

results['SVM_Landmarks'] = {
    'model': svm_landmarks,
    'train_acc': accuracy_score(y_train, y_pred_train),
    'test_acc': accuracy_score(y_test, y_pred_test),
    'train_f1': f1_score(y_train, y_pred_train, average='macro'),
    'test_f1': f1_score(y_test, y_pred_test, average='macro'),
    'train_time': train_time,
    'y_pred': y_pred_test,
    'feature_type': 'Landmarks'
}

print(f"  ✓ Training completed in {train_time:.2f}s")
print(f"  Train Accuracy: {results['SVM_Landmarks']['train_acc']:.4f}")
print(f"  Test Accuracy:  {results['SVM_Landmarks']['test_acc']:.4f}")
print(f"  Test Macro-F1:  {results['SVM_Landmarks']['test_f1']:.4f}")

# Save model
with open('model_svm_landmarks.pkl', 'wb') as f:
    pickle.dump(svm_landmarks, f)

# Model 2: Random Forest (Landmark features)
print("\n[2/4] Training Random Forest on Landmark features...")
print("  Hyperparameters: n_estimators=100, max_depth=20")
start_time = time.time()

rf_landmarks = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    random_state=RANDOM_SEED,
    n_jobs=-1,
    verbose=0
)
rf_landmarks.fit(X_train_landmarks, y_train)  # RF doesn't need standardization

train_time = time.time() - start_time
y_pred_train = rf_landmarks.predict(X_train_landmarks)
y_pred_test = rf_landmarks.predict(X_test_landmarks)

results['RF_Landmarks'] = {
    'model': rf_landmarks,
    'train_acc': accuracy_score(y_train, y_pred_train),
    'test_acc': accuracy_score(y_test, y_pred_test),
    'train_f1': f1_score(y_train, y_pred_train, average='macro'),
    'test_f1': f1_score(y_test, y_pred_test, average='macro'),
    'train_time': train_time,
    'y_pred': y_pred_test,
    'feature_type': 'Landmarks'
}

print(f"  ✓ Training completed in {train_time:.2f}s")
print(f"  Train Accuracy: {results['RF_Landmarks']['train_acc']:.4f}")
print(f"  Test Accuracy:  {results['RF_Landmarks']['test_acc']:.4f}")
print(f"  Test Macro-F1:  {results['RF_Landmarks']['test_f1']:.4f}")

# Save model
with open('model_rf_landmarks.pkl', 'wb') as f:
    pickle.dump(rf_landmarks, f)

# ============================================================================
# TRACK B: CNN FEATURES
# ============================================================================

print("\n" + "-"*60)
print("TRACK B: CNN DEEP FEATURES")
print("-"*60)

# Standardize features for CNN-based models
print("\nStandardizing CNN features...")
scaler_cnn = StandardScaler()
X_train_cnn_scaled = scaler_cnn.fit_transform(X_train_cnn)
X_test_cnn_scaled = scaler_cnn.transform(X_test_cnn)
print("✓ Features standardized")

# Save scaler
with open('scaler_cnn.pkl', 'wb') as f:
    pickle.dump(scaler_cnn, f)

# Model 3: SVM with RBF kernel (CNN features)
print("\n[3/4] Training SVM on CNN features...")
print("  Hyperparameters: C=10.0, kernel='rbf', gamma='scale'")
start_time = time.time()

svm_cnn = SVC(
    C=10.0,
    kernel='rbf',
    gamma='scale',
    random_state=RANDOM_SEED,
    verbose=False
)
svm_cnn.fit(X_train_cnn_scaled, y_train_cnn)

train_time = time.time() - start_time
y_pred_train = svm_cnn.predict(X_train_cnn_scaled)
y_pred_test = svm_cnn.predict(X_test_cnn_scaled)

results['SVM_CNN'] = {
    'model': svm_cnn,
    'train_acc': accuracy_score(y_train_cnn, y_pred_train),
    'test_acc': accuracy_score(y_test_cnn, y_pred_test),
    'train_f1': f1_score(y_train_cnn, y_pred_train, average='macro'),
    'test_f1': f1_score(y_test_cnn, y_pred_test, average='macro'),
    'train_time': train_time,
    'y_pred': y_pred_test,
    'feature_type': 'CNN'
}

print(f"  ✓ Training completed in {train_time:.2f}s")
print(f"  Train Accuracy: {results['SVM_CNN']['train_acc']:.4f}")
print(f"  Test Accuracy:  {results['SVM_CNN']['test_acc']:.4f}")
print(f"  Test Macro-F1:  {results['SVM_CNN']['test_f1']:.4f}")

# Save model
with open('model_svm_cnn.pkl', 'wb') as f:
    pickle.dump(svm_cnn, f)

# Model 4: Logistic Regression (CNN features)
print("\n[4/4] Training Logistic Regression on CNN features...")
print("  Hyperparameters: C=1.0, max_iter=1000, multi_class='multinomial'")
start_time = time.time()

logreg_cnn = LogisticRegression(
    C=1.0,
    max_iter=1000,
    multi_class='multinomial',
    random_state=RANDOM_SEED,
    verbose=0,
    n_jobs=-1
)
logreg_cnn.fit(X_train_cnn_scaled, y_train_cnn)

train_time = time.time() - start_time
y_pred_train = logreg_cnn.predict(X_train_cnn_scaled)
y_pred_test = logreg_cnn.predict(X_test_cnn_scaled)

results['LogReg_CNN'] = {
    'model': logreg_cnn,
    'train_acc': accuracy_score(y_train_cnn, y_pred_train),
    'test_acc': accuracy_score(y_test_cnn, y_pred_test),
    'train_f1': f1_score(y_train_cnn, y_pred_train, average='macro'),
    'test_f1': f1_score(y_test_cnn, y_pred_test, average='macro'),
    'train_time': train_time,
    'y_pred': y_pred_test,
    'feature_type': 'CNN'
}

print(f"  ✓ Training completed in {train_time:.2f}s")
print(f"  Train Accuracy: {results['LogReg_CNN']['train_acc']:.4f}")
print(f"  Test Accuracy:  {results['LogReg_CNN']['test_acc']:.4f}")
print(f"  Test Macro-F1:  {results['LogReg_CNN']['test_f1']:.4f}")

# Save model
with open('model_logreg_cnn.pkl', 'wb') as f:
    pickle.dump(logreg_cnn, f)

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)

summary_data = []
for model_name, result in results.items():
    summary_data.append({
        'Model': model_name,
        'Features': result['feature_type'],
        'Train Acc': f"{result['train_acc']:.4f}",
        'Test Acc': f"{result['test_acc']:.4f}",
        'Test F1': f"{result['test_f1']:.4f}",
        'Time (s)': f"{result['train_time']:.2f}"
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))
print("="*60)

# Identify best model
best_model_name = max(results.keys(), key=lambda k: results[k]['test_acc'])
best_model_info = results[best_model_name]

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   Test Accuracy: {best_model_info['test_acc']:.4f}")
print(f"   Test Macro-F1: {best_model_info['test_f1']:.4f}")
print("="*60)

# Save all results
with open('training_results.pkl', 'wb') as f:
    pickle.dump(results, f)
print("\n✓ All models and results saved")

In [ ]:
# Milestone 3: Evaluation & Reflection
from sklearn.metrics import (classification_report, confusion_matrix, 
                             precision_score, recall_score, f1_score)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

print("\n" + "="*60)
print("DETAILED EVALUATION")
print("="*60)

# Function to evaluate a single model
def evaluate_model(model_name, y_true, y_pred, emotions):
    """Comprehensive evaluation of a single model"""
    print("\n" + "="*60)
    print(f"MODEL: {model_name}")
    print("="*60)
    
    # Overall metrics
    accuracy = accuracy_score(y_true, y_pred)
    macro_precision = precision_score(y_true, y_pred, average='macro')
    macro_recall = recall_score(y_true, y_pred, average='macro')
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    
    print(f"\n📊 OVERALL METRICS:")
    print(f"  Accuracy:        {accuracy:.4f}")
    print(f"  Macro Precision: {macro_precision:.4f}")
    print(f"  Macro Recall:    {macro_recall:.4f}")
    print(f"  Macro F1-Score:  {macro_f1:.4f}")
    
    # Classification report
    print(f"\n📋 CLASSIFICATION REPORT:")
    print("-"*60)
    report = classification_report(y_true, y_pred, 
                                   target_names=emotions,
                                   digits=4)
    print(report)
    
    # Get per-class metrics for analysis
    report_dict = classification_report(y_true, y_pred, 
                                       target_names=emotions,
                                       output_dict=True)
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
               xticklabels=emotions, yticklabels=emotions,
               cbar_kws={'label': 'Count'})
    plt.title(f'Confusion Matrix - {model_name}', fontsize=14, fontweight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    # Analyze confusion patterns
    print(f"\n🔍 CONFUSION ANALYSIS:")
    print("-"*60)
    
    # Normalize confusion matrix by row (true labels)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Find top confusions (excluding diagonal)
    confusions = []
    for i in range(len(emotions)):
        for j in range(len(emotions)):
            if i != j and cm[i, j] > 0:
                confusions.append({
                    'true': emotions[i],
                    'pred': emotions[j],
                    'count': cm[i, j],
                    'rate': cm_normalized[i, j]
                })
    
    # Sort by count
    confusions = sorted(confusions, key=lambda x: x['count'], reverse=True)
    
    print("Top confusion patterns:")
    for conf in confusions[:5]:
        print(f"  • {conf['true']:8s} → {conf['pred']:8s}: "
              f"{conf['count']:3d} errors ({conf['rate']*100:.1f}%)")
    
    # Identify hardest classes (lowest F1)
    class_f1 = [(emotions[i], report_dict[emotions[i]]['f1-score']) 
                for i in range(len(emotions))]
    class_f1 = sorted(class_f1, key=lambda x: x[1])
    
    print(f"\n⚠️  HARDEST CLASSES (by F1-score):")
    for emotion, f1 in class_f1[:3]:
        precision = report_dict[emotion]['precision']
        recall = report_dict[emotion]['recall']
        support = report_dict[emotion]['support']
        print(f"  • {emotion:8s}: F1={f1:.4f} (P={precision:.4f}, "
              f"R={recall:.4f}, N={support:.0f})")
    
    return {
        'accuracy': accuracy,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'confusion_matrix': cm,
        'report_dict': report_dict,
        'top_confusions': confusions[:5],
        'hardest_classes': class_f1[:3]
    }

# Evaluate all models
evaluation_results = {}

for model_name, model_info in results.items():
    if model_info['feature_type'] == 'Landmarks':
        y_true = y_test
    else:
        y_true = y_test_cnn
    
    eval_result = evaluate_model(model_name, y_true, 
                                 model_info['y_pred'], EMOTIONS)
    evaluation_results[model_name] = eval_result

# ============================================================================
# COMPARATIVE SUMMARY
# ============================================================================

print("\n" + "="*60)
print("COMPARATIVE SUMMARY")
print("="*60)

# Create comparison table
comparison_data = []
for model_name in results.keys():
    eval_res = evaluation_results[model_name]
    comparison_data.append({
        'Model': model_name,
        'Accuracy': f"{eval_res['accuracy']:.4f}",
        'Macro-Precision': f"{eval_res['macro_precision']:.4f}",
        'Macro-Recall': f"{eval_res['macro_recall']:.4f}",
        'Macro-F1': f"{eval_res['macro_f1']:.4f}"
    })

comparison_df = pd.DataFrame(comparison_data)
print("\n" + comparison_df.to_string(index=False))

# ============================================================================
# KEY INSIGHTS
# ============================================================================

print("\n" + "="*60)
print("KEY INSIGHTS")
print("="*60)

# 1. Best performing model
best_model = max(evaluation_results.keys(), 
                key=lambda k: evaluation_results[k]['accuracy'])
best_acc = evaluation_results[best_model]['accuracy']

print(f"\n1. 🏆 BEST MODEL: {best_model}")
print(f"   • Test Accuracy: {best_acc:.4f}")
print(f"   • Feature Type: {results[best_model]['feature_type']}")
if results[best_model]['feature_type'] == 'CNN':
    print(f"   • Reason: Deep features from pretrained CNN capture")
    print(f"     discriminative patterns better than hand-crafted landmarks")
else:
    print(f"   • Reason: Facial landmark geometry effectively captures")
    print(f"     emotion-specific facial configurations")

# 2. Hardest classes across all models
print(f"\n2. ⚠️  HARDEST CLASSES (most commonly confused):")
all_hardest = {}
for model_name, eval_res in evaluation_results.items():
    for emotion, f1 in eval_res['hardest_classes']:
        if emotion not in all_hardest:
            all_hardest[emotion] = []
        all_hardest[emotion].append(f1)

avg_f1_by_class = {k: np.mean(v) for k, v in all_hardest.items()}
hardest_overall = sorted(avg_f1_by_class.items(), key=lambda x: x[1])[:3]

for emotion, avg_f1 in hardest_overall:
    print(f"   • {emotion:8s}: Average F1 = {avg_f1:.4f}")
    # Get common confusions for this class
    for model_name, eval_res in evaluation_results.items():
        for conf in eval_res['top_confusions']:
            if conf['true'] == emotion:
                print(f"     └─ Often confused with '{conf['pred']}' "
                      f"({conf['count']} times in {model_name})")
                break

# 3. Feature comparison
print(f"\n3. 📊 FEATURE TYPE COMPARISON:")
landmark_models = [k for k, v in results.items() if v['feature_type'] == 'Landmarks']
cnn_models = [k for k, v in results.items() if v['feature_type'] == 'CNN']

landmark_avg = np.mean([evaluation_results[k]['accuracy'] for k in landmark_models])
cnn_avg = np.mean([evaluation_results[k]['accuracy'] for k in cnn_models])

print(f"   • Landmark features: Avg accuracy = {landmark_avg:.4f}")
print(f"   • CNN features:      Avg accuracy = {cnn_avg:.4f}")
print(f"   • Difference:        {abs(cnn_avg - landmark_avg):.4f}")

# 4. Common confusion patterns
print(f"\n4. 🔄 MOST COMMON CONFUSION PATTERNS:")
all_confusions = {}
for model_name, eval_res in evaluation_results.items():
    for conf in eval_res['top_confusions']:
        key = f"{conf['true']} → {conf['pred']}"
        if key not in all_confusions:
            all_confusions[key] = 0
        all_confusions[key] += conf['count']

top_confusions = sorted(all_confusions.items(), key=lambda x: x[1], reverse=True)[:3]
for pattern, total_count in top_confusions:
    print(f"   • {pattern}: {total_count} total errors across all models")

# 5. Potential improvements
print(f"\n5. 💡 POTENTIAL IMPROVEMENTS:")
print(f"   • Data Augmentation: Apply rotation, flipping, brightness")
print(f"     adjustments to increase training diversity")
print(f"   • Ensemble Methods: Combine predictions from multiple models")
print(f"   • Fine-tuning CNN: Train the CNN backbone on FER2013 instead")
print(f"     of just using as feature extractor")
print(f"   • Address class imbalance: Use weighted loss or oversampling")
print(f"   • Deeper architecture: Try ResNet or EfficientNet for features")

print("\n" + "="*60)

# Save evaluation results
with open('evaluation_results.pkl', 'wb') as f:
    pickle.dump(evaluation_results, f)
print("\n✓ Evaluation results saved")

In [ ]:
# Misclassification Analysis
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

def analyze_misclassifications(model_name, y_true, y_pred, 
                               data_dir, emotions, num_examples=6):
    """Analyze and visualize misclassified examples"""
    print("\n" + "="*60)
    print(f"MISCLASSIFICATION ANALYSIS - {model_name}")
    print("="*60)
    
    # Find misclassified indices
    misclassified_indices = np.where(y_true != y_pred)[0]
    print(f"\nTotal misclassifications: {len(misclassified_indices)}")
    print(f"Error rate: {len(misclassified_indices)/len(y_true)*100:.2f}%")
    
    # Get all image paths
    all_image_paths = []
    all_labels = []
    
    for emotion_idx, emotion in enumerate(emotions):
        emotion_path = os.path.join(data_dir, emotion)
        if not os.path.exists(emotion_path):
            continue
        
        images = sorted([f for f in os.listdir(emotion_path) 
                        if f.endswith(('.jpg', '.png', '.jpeg'))])
        
        for img_name in images:
            all_image_paths.append(os.path.join(emotion_path, img_name))
            all_labels.append(emotion_idx)
    
    # Sample interesting misclassifications
    # Prioritize: high confidence errors, diverse error types
    misclass_info = []
    for idx in misclassified_indices:
        true_label = y_true[idx]
        pred_label = y_pred[idx]
        misclass_info.append({
            'idx': idx,
            'true': true_label,
            'pred': pred_label,
            'true_name': emotions[true_label],
            'pred_name': emotions[pred_label]
        })
    
    # Sample diverse examples (different error types)
    unique_errors = {}
    for info in misclass_info:
        key = (info['true'], info['pred'])
        if key not in unique_errors:
            unique_errors[key] = []
        unique_errors[key].append(info)
    
    # Select examples
    selected_examples = []
    for key, examples in unique_errors.items():
        if len(selected_examples) < num_examples:
            selected_examples.extend(examples[:1])
    
    # Fill remaining slots with random samples
    if len(selected_examples) < num_examples:
        remaining = [e for e in misclass_info if e not in selected_examples]
        np.random.shuffle(remaining)
        selected_examples.extend(remaining[:num_examples - len(selected_examples)])
    
    selected_examples = selected_examples[:num_examples]
    
    # Visualize
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    fig.suptitle(f'Misclassified Examples - {model_name}', 
                fontsize=16, fontweight='bold')
    
    for i, example in enumerate(selected_examples):
        img_path = all_image_paths[example['idx']]
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        axes[i].imshow(img, cmap='gray')
        axes[i].axis('off')
        
        title = (f"True: {example['true_name']}\n"
                f"Predicted: {example['pred_name']}")
        axes[i].set_title(title, fontsize=10, color='red', fontweight='bold')
        
        # Analysis comment
        comment = analyze_error_cause(example['true_name'], 
                                      example['pred_name'])
        axes[i].text(0.5, -0.15, comment, 
                    transform=axes[i].transAxes,
                    ha='center', fontsize=8, 
                    style='italic', wrap=True)
    
    # Hide unused subplots
    for i in range(len(selected_examples), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed analysis
    print(f"\n🔍 EXAMPLE ANALYSIS:")
    print("-"*60)
    for i, example in enumerate(selected_examples, 1):
        print(f"\n{i}. {example['true_name']} → {example['pred_name']}")
        print(f"   Likely cause: {analyze_error_cause(example['true_name'], example['pred_name'])}")

def analyze_error_cause(true_emotion, pred_emotion):
    """Provide likely cause for misclassification"""
    # Common confusion patterns and their causes
    confusions = {
        ('happy', 'neutral'): "Subtle smile, weak expression intensity",
        ('neutral', 'sad'): "Relaxed face resembles slight sadness",
        ('fear', 'surprise'): "Similar eye widening, mouth opening",
        ('angry', 'disgust'): "Shared eyebrow lowering, nose wrinkling",
        ('sad', 'neutral'): "Mild sadness hard to distinguish from neutral",
        ('surprise', 'fear'): "Both show wide eyes, raised eyebrows",
        ('disgust', 'angry'): "Similar facial muscle tension patterns",
        ('happy', 'surprise'): "Mouth opening in both expressions",
        ('neutral', 'happy'): "Very subtle positive expression",
        ('fear', 'sad'): "Both show eye tension and downturned features"
    }
    
    key = (true_emotion, pred_emotion)
    if key in confusions:
        return confusions[key]
    else:
        return "Ambiguous expression or poor image quality"

# Analyze misclassifications for best model
best_model_name = max(results.keys(), key=lambda k: results[k]['test_acc'])

if results[best_model_name]['feature_type'] == 'Landmarks':
    y_true_best = y_test
    data_dir_best = TEST_DIR
else:
    y_true_best = y_test_cnn
    data_dir_best = TEST_DIR

analyze_misclassifications(
    best_model_name,
    y_true_best,
    results[best_model_name]['y_pred'],
    data_dir_best,
    EMOTIONS,
    num_examples=6
)

# Compare misclassifications across models
print("\n" + "="*60)
print("MISCLASSIFICATION COMPARISON")
print("="*60)

for model_name, model_info in results.items():
    if model_info['feature_type'] == 'Landmarks':
        y_true_comp = y_test
    else:
        y_true_comp = y_test_cnn
    
    misclass_count = np.sum(y_true_comp != model_info['y_pred'])
    error_rate = misclass_count / len(y_true_comp) * 100
    
    print(f"\n{model_name}:")
    print(f"  Misclassifications: {misclass_count}/{len(y_true_comp)}")
    print(f"  Error rate: {error_rate:.2f}%")

print("\n" + "="*60)

In [ ]:
# Simple Deployment - Predict Function
import cv2
import numpy as np
import pickle
import matplotlib.pyplot as plt

class EmotionPredictor:
    """Complete emotion prediction pipeline"""
    
    def __init__(self, model_path, scaler_path, feature_extractor, 
                 emotions, feature_type='CNN'):
        """
        Initialize emotion predictor
        
        Args:
            model_path: Path to saved model (.pkl)
            scaler_path: Path to saved scaler (.pkl)
            feature_extractor: Feature extraction object
            emotions: List of emotion labels
            feature_type: 'CNN' or 'Landmarks'
        """
        self.emotions = emotions
        self.feature_type = feature_type
        self.feature_extractor = feature_extractor
        
        # Load model
        with open(model_path, 'rb') as f:
            self.model = pickle.load(f)
        print(f"✓ Loaded model from {model_path}")
        
        # Load scaler
        with open(scaler_path, 'rb') as f:
            self.scaler = pickle.load(f)
        print(f"✓ Loaded scaler from {scaler_path}")
    
    def predict(self, image_path):
        """
        Predict emotion from image file
        
        Args:
            image_path: Path to image file
            
        Returns:
            dict with prediction results
        """
        # Read image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f"Could not read image: {image_path}")
        
        # Extract features
        features = self.feature_extractor.extract_features(image)
        
        if features is None:
            return {
                'success': False,
                'error': 'Could not extract features (no face detected)',
                'image': image
            }
        
        # Standardize features
        features_scaled = self.scaler.transform(features.reshape(1, -1))
        
        # Predict
        prediction = self.model.predict(features_scaled)[0]
        
        # Get prediction probabilities if available
        if hasattr(self.model, 'predict_proba'):
            probabilities = self.model.predict_proba(features_scaled)[0]
        elif hasattr(self.model, 'decision_function'):
            # For SVM, convert decision function to pseudo-probabilities
            decision = self.model.decision_function(features_scaled)[0]
            # Softmax normalization
            exp_decision = np.exp(decision - np.max(decision))
            probabilities = exp_decision / exp_decision.sum()
        else:
            probabilities = None
        
        return {
            'success': True,
            'emotion': self.emotions[prediction],
            'emotion_idx': prediction,
            'probabilities': probabilities,
            'image': image,
            'features': features
        }
    
    def predict_and_visualize(self, image_path):
        """Predict and visualize result"""
        result = self.predict(image_path)
        
        if not result['success']:
            print(f"❌ Prediction failed: {result['error']}")
            plt.figure(figsize=(6, 6))
            plt.imshow(result['image'], cmap='gray')
            plt.title(f"Error: {result['error']}", color='red')
            plt.axis('off')
            plt.show()
            return result
        
        # Visualize
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Image with prediction
        axes[0].imshow(result['image'], cmap='gray')
        axes[0].set_title(f"Predicted: {result['emotion']}", 
                         fontsize=14, fontweight='bold', color='green')
        axes[0].axis('off')
        
        # Probability bar chart
        if result['probabilities'] is not None:
            axes[1].barh(self.emotions, result['probabilities'], color='skyblue')
            axes[1].axvline(x=result['probabilities'][result['emotion_idx']], 
                           color='green', linestyle='--', linewidth=2)
            axes[1].set_xlabel('Probability', fontsize=12)
            axes[1].set_title('Emotion Probabilities', fontsize=14, fontweight='bold')
            axes[1].set_xlim([0, 1])
            
            # Highlight predicted class
            for i, (emotion, prob) in enumerate(zip(self.emotions, result['probabilities'])):
                if i == result['emotion_idx']:
                    axes[1].get_yticklabels()[i].set_weight('bold')
                    axes[1].get_yticklabels()[i].set_color('green')
        else:
            axes[1].text(0.5, 0.5, 'Probabilities not available', 
                        ha='center', va='center', fontsize=12)
            axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n{'='*60}")
        print(f"PREDICTION RESULT")
        print(f"{'='*60}")
        print(f"Image: {image_path}")
        print(f"Predicted Emotion: {result['emotion']}")
        if result['probabilities'] is not None:
            print(f"Confidence: {result['probabilities'][result['emotion_idx']]:.4f}")
            print(f"\nAll probabilities:")
            for emotion, prob in zip(self.emotions, result['probabilities']):
                print(f"  {emotion:10s}: {prob:.4f}")
        print(f"{'='*60}\n")
        
        return result

# ============================================================================
# Initialize predictor with best model
# ============================================================================

print("\n" + "="*60)
print("INITIALIZING EMOTION PREDICTOR")
print("="*60)

# Determine best model
best_model_name = max(results.keys(), key=lambda k: results[k]['test_acc'])
print(f"\nUsing best model: {best_model_name}")
print(f"Test Accuracy: {results[best_model_name]['test_acc']:.4f}")

# Set paths based on model type
if results[best_model_name]['feature_type'] == 'CNN':
    model_path = 'model_svm_cnn.pkl' if 'SVM' in best_model_name else 'model_logreg_cnn.pkl'
    scaler_path = 'scaler_cnn.pkl'
    feature_extractor = cnn_extractor
    feature_type = 'CNN'
else:
    model_path = 'model_svm_landmarks.pkl' if 'SVM' in best_model_name else 'model_rf_landmarks.pkl'
    scaler_path = 'scaler_landmarks.pkl'
    feature_extractor = landmark_extractor
    feature_type = 'Landmarks'

# Create predictor
predictor = EmotionPredictor(
    model_path=model_path,
    scaler_path=scaler_path,
    feature_extractor=feature_extractor,
    emotions=EMOTIONS,
    feature_type=feature_type
)

print(f"✓ Predictor ready (Feature type: {feature_type})")
print("="*60)

# ============================================================================
# Demonstrate on unseen images
# ============================================================================

print("\n" + "="*60)
print("TESTING ON UNSEEN IMAGES")
print("="*60)

# Select random images from test set for demonstration
def get_random_test_images(data_dir, emotions, num_images=2):
    """Get random test images"""
    selected_images = []
    
    for emotion in np.random.choice(emotions, num_images, replace=False):
        emotion_path = os.path.join(data_dir, emotion)
        if not os.path.exists(emotion_path):
            continue
        
        images = [f for f in os.listdir(emotion_path) 
                 if f.endswith(('.jpg', '.png', '.jpeg'))]
        
        if len(images) > 0:
            img_name = np.random.choice(images)
            img_path = os.path.join(emotion_path, img_name)
            selected_images.append((img_path, emotion))
    
    return selected_images

# Get test images
test_images = get_random_test_images(TEST_DIR, EMOTIONS, num_images=3)

# Predict on each image
for i, (img_path, true_emotion) in enumerate(test_images, 1):
    print(f"\n{'='*60}")
    print(f"TEST IMAGE {i}")
    print(f"{'='*60}")
    print(f"True emotion: {true_emotion}")
    
    result = predictor.predict_and_visualize(img_path)

# ============================================================================
# Save predictor for future use
# ============================================================================

print("\n" + "="*60)
print("SAVING PREDICTOR")
print("="*60)

with open('emotion_predictor.pkl', 'wb') as f:
    pickle.dump(predictor, f)

print("✓ Predictor saved to 'emotion_predictor.pkl'")
print("\nTo use in the future:")
print("  with open('emotion_predictor.pkl', 'rb') as f:")
print("      predictor = pickle.load(f)")
print("  result = predictor.predict('path/to/image.jpg')")
print("="*60)

In [ ]:
# Utility: Load Previously Extracted Features
# Use this if you've already extracted features and want to skip that step

import numpy as np

def load_saved_features(landmarks=True, cnn=True):
    """
    Load previously saved features
    
    Args:
        landmarks: Load landmark features
        cnn: Load CNN features
    
    Returns:
        Dictionary with loaded features
    """
    loaded = {}
    
    if landmarks:
        try:
            print("Loading landmark features...")
            train_data = np.load('features_train_landmarks.npz')
            test_data = np.load('features_test_landmarks.npz')
            
            loaded['X_train_landmarks'] = train_data['X']
            loaded['y_train'] = train_data['y']
            loaded['X_test_landmarks'] = test_data['X']
            loaded['y_test'] = test_data['y']
            loaded['emotions'] = train_data['emotions']
            
            print(f"✓ Landmark features loaded")
            print(f"  Train: {loaded['X_train_landmarks'].shape}")
            print(f"  Test:  {loaded['X_test_landmarks'].shape}")
        except FileNotFoundError:
            print("❌ Landmark feature files not found")
    
    if cnn:
        try:
            print("\nLoading CNN features...")
            train_data = np.load('features_train_cnn.npz')
            test_data = np.load('features_test_cnn.npz')
            
            loaded['X_train_cnn'] = train_data['X']
            loaded['y_train_cnn'] = train_data['y']
            loaded['X_test_cnn'] = test_data['X']
            loaded['y_test_cnn'] = test_data['y']
            
            print(f"✓ CNN features loaded")
            print(f"  Train: {loaded['X_train_cnn'].shape}")
            print(f"  Test:  {loaded['X_test_cnn'].shape}")
        except FileNotFoundError:
            print("❌ CNN feature files not found")
    
    return loaded

# Example usage:
# features = load_saved_features(landmarks=True, cnn=True)
# X_train_landmarks = features['X_train_landmarks']
# y_train = features['y_train']
# etc.

def load_saved_models():
    """Load all saved models and scalers"""
    import pickle
    
    loaded_models = {}
    
    model_files = {
        'SVM_Landmarks': ('model_svm_landmarks.pkl', 'scaler_landmarks.pkl'),
        'RF_Landmarks': ('model_rf_landmarks.pkl', None),
        'SVM_CNN': ('model_svm_cnn.pkl', 'scaler_cnn.pkl'),
        'LogReg_CNN': ('model_logreg_cnn.pkl', 'scaler_cnn.pkl')
    }
    
    for model_name, (model_path, scaler_path) in model_files.items():
        try:
            with open(model_path, 'rb') as f:
                model = pickle.load(f)
            
            scaler = None
            if scaler_path:
                with open(scaler_path, 'rb') as f:
                    scaler = pickle.load(f)
            
            loaded_models[model_name] = {
                'model': model,
                'scaler': scaler
            }
            print(f"✓ Loaded {model_name}")
        except FileNotFoundError:
            print(f"❌ Could not load {model_name}")
    
    return loaded_models

# Example usage:
# models = load_saved_models()
# svm_model = models['SVM_CNN']['model']
# svm_scaler = models['SVM_CNN']['scaler']